In [3]:
import os
from re import S
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
import librosa
import yaml

#your task
from model import RawNet
from biosegment import wav2bio

ImportError: attempted relative import with no known parent package

In [4]:
def get_Bio(X_pad, fs):

    bio = wav2bio(X_pad, fs)
    # bio_length = len(bio)
    bio_inp = torch.IntTensor(bio)
    bio_length = torch.IntTensor([len(bio)])
    return bio_inp, bio_length

def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

def parse_input(file_path):
    cut = 64600  # take ~4 sec audio (64600 samples)
    X, fs = librosa.load(file_path, sr=16000)
    X_pad = pad(X, cut)
    x_inp = Tensor(X_pad)
    bio_inp, bio_length = get_Bio(X_pad, fs)
    
        
    return x_inp.unsqueeze(0).to(device), bio_inp.unsqueeze(0).to(device), bio_length.to(device)

In [5]:
config_path = "configs/model_config_RawNet_Trans_32concat.yaml"
model_path = "models/trans_32concat/epoch_91.pth"
device = 'cuda:0' 
with open(config_path, 'r') as f_yaml:
    parser1 = yaml.safe_load(f_yaml)

model = RawNet(parser1['model'], device).to(device)
model.load_state_dict(torch.load(model_path,map_location=device))

NameError: name 'RawNet' is not defined

In [16]:
# path = "/root/biological/asvspoof2021/LA/Baseline-RawNet2-bio/Demo/prof.wav"
# y, sr = librosa.load(path,sr = 44100)
# y_16k = librosa.resample(y, orig_sr=sr, target_sr=16000)
# import soundfile as sf
# sf.write('/root/biological/asvspoof2021/LA/Baseline-RawNet2-bio/Demo/prof_16k.flac', y_16k, 16000, format='flac', subtype='PCM_24')

In [17]:
model.eval()
# Load the audio file that we need to check whether it is fake voice or not.
wav = "/root/dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_5085671.flac"
# wav = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_5464494.flac"
# wav = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_2925019.flac"

# wav = "/root/dataset/Speech/AIHub_Studio/wavs/01FHSH0028/01FHSH0028_00122.wav"

# wav = "/root/biological/asvspoof2021/LA/Baseline-RawNet2-bio/Demo/prof_16k.flac"

# Extract the feature of the audio
x_inp, bio_inp, bio_length = parse_input(wav)

# use the model to calculate the prediction value
out, _ = model(x_inp, bio_inp, bio_length)

# out = [0.123345135,-0.1234145124]

# Convert into readable format
_, pred = out.max(dim=1)
if pred:
    print("bonafide") #real voice
else:
    print("spoof") #fake voice

spoof


tensor([-1.5497e-05], grad_fn=<MaxBackward0>)
tensor([0])


In [44]:
out

tensor([[-1.0610e-05, -1.1457e+01]], grad_fn=<LogSoftmaxBackward>)

In [45]:
model.eval()

RawNet(
  (Sinc_conv): SincConv()
  (first_bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (selu): SELU(inplace=True)
  (block0): Sequential(
    (0): Residual_block(
      (lrelu): LeakyReLU(negative_slope=0.3)
      (conv1): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (mp): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (block1): Sequential(
    (0): Residual_block(
      (bn1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (lrelu): LeakyReLU(negative_slope=0.3)
      (conv1): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(20, 20, kernel_size=(3,), stride

In [48]:
config_path = "configs/model_config_RawNet_Trans_32concat.yaml"
model_path = "models/trans_32concat/epoch_91.pth"
device = 'cpu' 
with open(config_path, 'r') as f_yaml:
    parser1 = yaml.safe_load(f_yaml)
model = RawNet(parser1['model'], device)
model.load_state_dict(torch.load(model_path,map_location=device))

<All keys matched successfully>